In [ ]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score
import os
from google.colab import drive
import numpy as np
import ast

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


# Load CSV files
gpt35 = pd.read_csv('/content/drive/My Drive/Lavoro ChatGPT/dataset/Sebastiano/13/13_GoEmo_New_with_answers_GPT3.5.csv')

gpt4 = pd.read_csv('/content/drive/My Drive/Lavoro ChatGPT/dataset/Sebastiano/13/13_GoEmo_New_with_answers_GPT4.csv')


In [ ]:


annotation_mapping = {'admiration':0, 'amusement':1, 'anger':2, 'annoyance':3, 'approval':4, 'caring':5, 'confusion':6, 'curiosity':7, 'desire':8, 'disappointment':9, 'disapproval':10, 'disgust':11, 'embarrassment':12, 'excitement':13, 'fear':14, 'gratitude':15, 'grief':16, 'joy':17, 'love':18, 'nervousness':19, 'optimism':20, 'pride':21, 'realization':22, 'relief':23, 'remorse':24, 'sadness':25, 'surprise':26, 'neutral':27}


def map_annotation(annotation_list):
      #print (string for string in annotation_list)
      return [annotation_mapping[string] for string in annotation_list if string in annotation_mapping]

def map_label(annotation_str):

    if ',' in annotation_str:
      return [annotation_mapping[string.strip()] for string in annotation_str.split(',')]
    else:
      return [annotation_mapping[annotation_str.strip()]]







In [ ]:
gpt4.head()

In [ ]:
gpt35['annotation_int'] = gpt35['annotation'].apply(lambda x: map_label(x))
gpt4['annotation_int'] = gpt4['annotation'].apply(lambda x: map_label(x))


answer_columns35 = ['answerPrompt2', 'answerPrompt4', 'answerPrompt6', 'answerPrompt8', 'answerPrompt10']
answer_columns4 = ['answerPrompt2', 'answerPrompt4', 'answerPrompt6', 'answerPrompt8', 'answerPrompt10', 'answerOLD']

for system in answer_columns35:
  gpt35[system+'_int'] = gpt35[system].apply(lambda x: map_annotation(ast.literal_eval(x)))

for system in answer_columns4:
  gpt4[system+'_int'] = gpt4[system].apply(lambda x: map_annotation(ast.literal_eval(x)))

In [ ]:

# Iterate through answer columns
answer_columns = ['answerPrompt2_int', 'answerPrompt4_int', 'answerPrompt6_int', 'answerPrompt8_int', 'answerPrompt10_int', 'answerOLD_int']

results_df = pd.DataFrame(columns=['Run', 'F1_Macro_gpt35', 'Accuracy_gpt35', 'F1_Macro_gpt4', 'Accuracy_gpt4'])
#
mlb = MultiLabelBinarizer()

for system in answer_columns:
    true_labels = mlb.fit_transform(gpt35['annotation_int'])

    if system!='answerOLD_int':
      predicted_labels = mlb.transform(gpt35[system])
      f1_macro_gpt35 = f1_score(true_labels, predicted_labels, average='macro')
      accuracy_gpt35 = accuracy_score(true_labels, predicted_labels)
    else:
      f1_macro_gpt35 = 0
      accuracy_gpt35 = 0

    # Compute F1 Macro and Accuracy for gpt4
    true_labels = mlb.fit_transform(gpt4['annotation_int'])
    predicted_labels = mlb.transform(gpt4[system])
    f1_macro_gpt4 = f1_score(true_labels, predicted_labels, average='macro')
    accuracy_gpt4 = accuracy_score(true_labels, predicted_labels)

    new_row = pd.DataFrame([{
        'Run': system,
        'F1_Macro_gpt35': f1_macro_gpt35,
        'Accuracy_gpt35': accuracy_gpt35,
        'F1_Macro_gpt4': f1_macro_gpt4,
        'Accuracy_gpt4': accuracy_gpt4
    }])

    results_df = pd.concat([results_df, new_row], ignore_index=True)

print(results_df)

# Save results to a CSV file
results_df.to_csv('/content/drive/My Drive/Lavoro ChatGPT/dataset/Sebastiano/13/13_GoEmo_results.csv', index=False)



                  Run  F1_Macro_gpt35  Accuracy_gpt35  F1_Macro_gpt4  \
0   answerPrompt2_int        0.323682           0.157       0.322116   
1   answerPrompt4_int        0.361265           0.167       0.329726   
2   answerPrompt6_int        0.325544           0.188       0.356227   
3   answerPrompt8_int        0.337339           0.194       0.361964   
4  answerPrompt10_int        0.306425           0.185       0.356371   
5       answerOLD_int        0.000000           0.000       0.278104   

   Accuracy_gpt4  
0          0.245  
1          0.248  
2          0.257  
3          0.263  
4          0.261  
5          0.230  
